In [1]:
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import random
from core import read_dom_table
import seaborn as sns

In [2]:
if 'uniref50_chunks' not in os.listdir('../data/interim'):
    os.mkdir('../data/interim/uniref50_chunks/')

Download uniref 50 from uniprot

In [ ]:
if 'uniref50.faa.split' not in os.listdir('../data/interim/uniref50_chunks'):
    !conda run -n beaker seqkit split2 ../data/interim/uniref50_chunks/uniref50.faa -s 100000

In [15]:
if 'defense_finder.hmm' not in os.listdir('../data/interim'):
    os.system('cat ~/defense-finder-models/profiles/*.hmm > ../data/interim/defense_finder.hmm')

0

In [2]:
split_dir = '../data/interim/uniref50_chunks/uniref50.faa.split/'

In [2]:
df_uniref_out = '../data/interim/df_uniref_out/'

In [9]:
if 'df_uniref_out' not in os.listdir('../data/interim'):
    os.mkdir(df_uniref_out)

In [ ]:
def get_hmm_desc(model):
    rand = random.randint(0, 1_000_000_000)
    temp_out = '../data/interim/temp_hmm_stat' + str(rand) + '.txt'
    system, gene = model.split('__')
    gene = gene.split('.')[0]
    os.system(' '.join(['conda run -n beaker', 'hmmstat', '~/defense-finder-models/profiles/' + model, 
                        '>', temp_out]))
    stat_df = pd.read_csv(temp_out, 
                          sep='\s+', names=['idx', 'name', 'accession', 'nseq', 'eff_nseq', 
                                            'M', 'relent', 'info', 'p relE', 'compKL'], comment='#')
    os.system('rm ' + temp_out)
    stat_df['system'] = system
    stat_df['gene'] = gene
    stat_df['description'] = stat_df.apply(lambda row: ' '.join(row[['system', 'gene', 'name']].to_list()) if row['gene'] != row['name'] else
                                           ' '.join(row[['system', 'gene']].to_list()), axis=1)
    return stat_df

In [ ]:
df_info_list = Parallel(n_jobs=48)(delayed(get_hmm_desc)(model)
                                   for model in 
                                   tqdm(os.listdir('/home/gridsan/pdeweirdt/defense-finder-models/profiles/'), 
                                        position=0) if '.hmm' in model)
df_info_df = pd.concat(df_info_list)

100%|██████████| 1018/1018 [00:38<00:00, 26.30it/s]


In [ ]:
merged_top_genes = (top_genes.merge(df_info_df.rename(columns={'name': 'query'}), 
                                    on='query',
                                    how='inner'))
gene_map = dict()
for _, row in merged_top_genes.iterrows():
    k = row['target']
    v = row['description'] # may have to adjust
    gene_map[k] = 